In [1]:
import os
os.chdir("../")

In [2]:
import numpy as np
import torch

In [3]:
from storage.har_datasets import WISDMDataset, sts_medoids
from s3ts.api.dms.har_datasets import LDFDataset, DFDataset
from storage.label_mappings import *
from s3ts.api.nets.methods import create_model_from_DM, train_model, test_model

In [4]:
from torchvision.transforms import Normalize

In [5]:
# this dataset comes at 20hz
ds = WISDMDataset("./datasets/WISDM/", wsize=40, normalize=True, label_mapping=None)

In [6]:
if not os.path.exists("./datasets/WISDM/meds.npz"):
    meds = sts_medoids(ds, n=500)
    with open("./datasets/WISDM/meds.npz", "wb") as f:
        np.save(f, meds)
else:
    meds = np.load("./datasets/WISDM/meds.npz")

In [7]:
dfds = DFDataset(ds, patterns=meds, w=0.1, dm_transform=None, ram=False)

Loading cached dissimilarity frames if available...


In [8]:
DM = []

np.random.seed(42)
for i in np.random.choice(np.arange(len(dfds)), 500):
    dm, _, _ = dfds[i]
    DM.append(dm)

DM = torch.stack(DM)

dm_transform = Normalize(mean=DM.mean(dim=[0, 2, 3]), std=DM.std(dim=[0, 2, 3]))

In [9]:
dfds.dm_transform = dm_transform

In [10]:
data_split = {
    "train": lambda x: x<1000000,
    "val": lambda x: (x>=1000000) * (x<1050000),
    "test": lambda x: x>=1050000
}

dm = LDFDataset(dfds, data_split=data_split, batch_size=64, random_seed=42, num_workers=8)

In [11]:
len(dm.ds_train) + len(dm.ds_val) + len(dm.ds_test)

1096768

In [12]:
model = create_model_from_DM(dm, name=None, 
        dsrc="img", arch="cnn", task="cls")

Input shape:  torch.Size([1, 6, 40, 40])
Latent shape:  torch.Size([1, 40, 2, 40])


In [ ]:
model, data = train_model(dm, model, max_epochs=2)
print(data)